**Transfer Learning Across Regions**

The code defines a neural network, loads simulated pre-trained weights, freezes the base layers, and fine-tunes only the final layer using a small simulated Egyptian dataset. This demonstrates transfer learning by adapting a pre-trained model to a new domain with limited data.

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the model
class EmissionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.base = nn.Sequential(
            nn.Linear(10, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.classifier = nn.Linear(32, 1)
    
    def forward(self, x):
        return self.classifier(self.base(x))

# Initialize model
model = EmissionModel()

# 1. Simulate loading pre-trained weights (French data)
print("Loading pre-trained weights (simulated)...")
pretrained_weights = {
    'base.0.weight': torch.randn(64, 10) * 0.1,
    'base.0.bias': torch.randn(64) * 0.1,
    'base.2.weight': torch.randn(32, 64) * 0.1,
    'base.2.bias': torch.randn(32) * 0.1,
    'classifier.weight': torch.randn(1, 32) * 0.1,
    'classifier.bias': torch.randn(1) * 0.1
}
model.load_state_dict(pretrained_weights)

# 2. Freeze the base layers (feature extractor)
print("Freezing base layers...")
for param in model.base.parameters():
    param.requires_grad = False

# 3. Set up for fine-tuning only the classifier
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
criterion = nn.MSELoss() 

# Simulate Egyptian dataset 
print("Preparing Egyptian dataset...")
X_egypt = torch.randn(100, 10)  
y_egypt = torch.randn(100, 1)   

egypt_dataset = TensorDataset(X_egypt, y_egypt)
egypt_loader = DataLoader(egypt_dataset, batch_size=16, shuffle=True)

# Fine-tuning loop
print("Starting fine-tuning...")
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, targets in egypt_loader:
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    epoch_loss = running_loss / len(egypt_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

print("Fine-tuning complete!")

# Verify which layers are trainable
print("\nParameter training status:")
for name, param in model.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

Loading pre-trained weights (simulated)...
Freezing base layers...
Preparing Egyptian dataset...
Starting fine-tuning...
Epoch 1/10, Loss: 1.0166
Epoch 2/10, Loss: 1.0562
Epoch 3/10, Loss: 1.0598
Epoch 4/10, Loss: 1.0154
Epoch 5/10, Loss: 0.9920
Epoch 6/10, Loss: 1.0768
Epoch 7/10, Loss: 1.1197
Epoch 8/10, Loss: 1.0087
Epoch 9/10, Loss: 1.0632
Epoch 10/10, Loss: 1.0427
Fine-tuning complete!

Parameter training status:
base.0.weight: requires_grad=False
base.0.bias: requires_grad=False
base.2.weight: requires_grad=False
base.2.bias: requires_grad=False
classifier.weight: requires_grad=True
classifier.bias: requires_grad=True
